## 词向量
### one-hot
      什么是one-hot编码？one-hot编码，又称独热编码、一位有效编码。其方法是使用N位状态寄存器来对N个状态进行编码，每个状态都有它独立的寄存器位，并且在任意时候，其中只有一位有效。　　　　　
    假如只有一个特征是离散值：

　　　　{sex：{male， female，other}}

    该特征总共有3个不同的分类值，此时需要3个bit位表示该特征是什么值，对应bit位为1的位置对应原来的特征的值（一般情况下可以将原始的特征的取值进行排序，以便于后期使用），此时得到独热码为{100}男性 ，{010}女性，{001}其他

    假如多个特征需要独热码编码，那么久按照上面的方法依次将每个特征的独热码拼接起来：

　　　　{sex：{male， female，other}}

　　　　{grade：{一年级， 二年级，三年级， 四年级}}

    此时对于输入为{sex：male； grade： 四年级}进行独热编码，可以首先将sex按照上面的进行编码得到{100}，然后按照grade进行编码为{0001}，那么两者连接起来得到最后的独热码{1000001}；
    优缺点分析

    优点：一是解决了分类器不好处理离散数据的问题，二是在一定程度上也起到了扩充特征的作用（上面样本特征数从2扩展到了7）

    缺点：在文本特征表示上有些缺点就非常突出了。首先，它是一个词袋模型，不考虑词与词之间的顺序（文本中词的顺序信息也是很重要的）；其次，它假设词与词相互独立（在大多数情况下，词与词是相互影响的）；最后，它得到的特征是离散稀疏的。
sklearn实现one-hot


    

In [6]:
from sklearn import preprocessing  
      
# enc.n_values_ is ：每个特征值的特征数目，第一个特征数目是2，第二个特征数目是3，第三个特征数目是4。
# enc.feature_indices_ is ：表明每个特征在one-hot向量中的坐标范围，0-2 是第一个特征，2-5就是第二个特征，5-9是第三个特征。
# 后面三个就是把特征值转换为 one-hot编码，我们可以对比结果看看one-hot差别。

enc = preprocessing.OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1],[1, 0, 2]])
print("enc.n_values_ is:",enc.n_values_)
print("enc.feature_indices_ is:",enc.feature_indices_)
print(enc.transform([[0, 1, 1]]).toarray()) #[[1. 0. 第0位   0. 1. 0.  第一位  0. 1. 0. 0. 第一位]]
print(enc.transform([[1, 1, 1]]).toarray())
print(enc.transform([[1, 2, 1]]).toarray())



enc.n_values_ is: [2 3 4]
enc.feature_indices_ is: [0 2 5 9]
[[1. 0. 0. 1. 0. 0. 1. 0. 0.]]
[[0. 1. 0. 1. 0. 0. 1. 0. 0.]]
[[0. 1. 0. 0. 1. 0. 1. 0. 0.]]


C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:100: DeprecationWarning: The ``n_values_`` attribute was deprecated in version 0.20 and will be removed 0.22.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:100: DeprecationWarning: The ``feature_indices_`` attribute was 

### word2vec 
    One-hot本质上是将词当做一个个孤立的原子单元（atomic unit）去处理的。显然，one-hot向量的维度等于词典的大小,存在离散、高维、稀疏的特点，这在动辄上万甚至百万词典的实际应用中，面临着巨大的维度灾难问题（The Curse of Dimensionality）。
    如何连续、低维、稠密的表示词向量那？2003年，Bengio等人发表了一篇开创性的文章：A neural probabilistic language model[3]。在这篇文章里，他们总结出了一套用神经网络建立统计语言模型的框架（Neural Network Language Model，以下简称NNLM），并首次提出了word embedding的概念（虽然没有叫这个名字），从而奠定了包括word2vec在内后续研究word representation learning的基础。

NNLM模型的基本思想可以概括如下：

    假定词表中的每一个word都对应着一个连续的特征向量；
    假定一个连续平滑的概率模型，输入一段词向量的序列，可以输出这段序列的联合概率；
    同时学习词向量的权重和概率模型里的参数。
整个模型的网络结构见下图：
![](img/NNLM.png)
我们可以将整个模型拆分成两部分加以理解：

    首先是一个线性的Embedding层。它将输入的N−1个one-hot词向量，通过一个共享的D×V的矩阵C，映射为N−1个分布式的词向量（distributed vector）。其中，V是词典的大小，D是Embedding向量的维度（一个先验参数）。C矩阵里存储了要学习的word vector。

    其次是一个简单的前向反馈神经网络g。它由一个tanh隐层和一个softmax输出层组成。通过将Embedding层输出的N−1个词向量映射为一个长度为V的概率分布向量，从而对词典中的word在输入context下的条件概率做出预估：
$$ p(\omega_i|\omega_1,\omega_2,...,\omega_{t-1})\approx f(\omega_i,\omega_{t-1},...,\omega_{t-n+1}) = g(\omega_i,C(\omega_{t-n+1}),...,C(\omega_{t-1})) $$

    我们可以通过最小化一个cross-entropy的正则化损失函数来调整模型的参数θ：    
$$ L(\theta) = \frac 1 T \sum\limits_t log f(\omega_i,\omega_{t-1},...,\omega_{t-n+1}) + R(\theta)$$

其中，模型的参数θ包括了Embedding层矩阵C的元素，和前向反馈神经网络模型g里的权重。


首先，原始的NNLM模型做如下改造：

    移除前向反馈神经网络中非线性的hidden layer，直接将中间层的Embedding layer与输出层的softmax layer连接；
    忽略上下文环境的序列信息：输入的所有词向量均汇总到同一个Embedding layer；
    将Future words纳入上下文环境

得到的模型称之为CBOW模型（Continuous Bag-of-Words Model），也是word2vec算法的第一个模型：
![](img/CBOW.png)
从数学上看，CBoW模型等价于一个词袋模型的向量乘以一个Embedding矩阵，从而得到一个连续的embedding向量。这也是CBoW模型名称的由来。

    1 输入层：上下文单词的onehot.  {假设单词向量空间dim为V，上下文单词个数为C}
    2 所有onehot分别乘以共享的输入权重矩阵W. {VN矩阵，N为自己设定的数，初始化权重矩阵W}
    3 所得的向量 {因为是onehot所以为向量} 相加求平均作为隐层向量, size为1N.
    4 乘以输出权重矩阵W' {NV}
    5 得到向量 {1V} 激活函数处理得到V-dim概率分布  {PS: 因为是onehot嘛，其中的每一维斗代表着一个单词}
    6 概率最大的index所指示的单词为预测出的中间词（target word）与true label的onehot做比较，误差越小越好（根据误差更新权重矩阵）
所以，需要定义loss function（一般为交叉熵代价函数），采用梯度下降算法更新W和W'。训练完毕后，输入层的每个单词与矩阵W相乘得到的向量的就是我们想要的词向量（word embedding），这个矩阵（所有单词的word embedding）也叫做look up table（其实聪明的你已经看出来了，其实这个look up table就是矩阵W自身），也就是说，任何一个单词的onehot乘以这个矩阵都将得到自己的词向量。有了look up table就可以免去训练过程直接查表得到单词的词向量了。


CBOW模型依然是从context对target word的预测中学习到词向量的表达。反过来，我们能否从target word对context的预测中学习到word vector呢？答案显然是可以的：这个模型被称为Skip-gram模型（名称源于该模型在训练时会对上下文环境里的word进行采样）。
![](img/Skip-Gram.png)
如果将Skip-gram模型的前向计算过程写成数学形式，我们得到：
$$ p(\omega_o|\omega_i) = \frac {e^{U_oV_i}} {\sum_j e^{U_jV_i}}$$
其中，Vi是Embedding层矩阵里的列向量，也被称为wi的input vector。Uj是softmax层矩阵里的行向量，也被称为wi
的output vector。

因此，Skip-gram模型的本质是计算输入word的input vector与目标word的output vector之间的余弦相似度，并进行softmax归一化。我们要学习的模型参数正是这两类词向量

下面的图中给出了一些我们的训练样本的例子。我们选定句子“The quick brown fox jumps over lazy dog”，设定我们的窗口大小为2（window_size=2），也就是说我们仅选输入词前后各两个词和输入词进行组合。下图中，蓝色代表input word，方框内代表位于窗口内的单词。Training Samples（输入， 输出）
![](img/demo.png)

我们的模型将会从每对单词出现的次数中习得统计结果。例如，我们的神经网络可能会得到更多类似（“中国“，”英国“）这样的训练样本对，而对于（”英国“，”蝈蝈“）这样的组合却看到的很少。因此，当我们的模型完成训练后，给定一个单词”中国“作为输入，输出的结果中”英国“或者”俄罗斯“要比”蝈蝈“被赋予更高的概率。

再次提醒，最终我们需要的是训练出来的权重矩阵。

References:

[1] :https://www.cnblogs.com/guoyaohua/p/9240336.html

[2] :https://www.jianshu.com/p/471d9bfbd72f

[3] :https://blog.csdn.net/lzc4869/article/details/79493427


In [6]:
#!/bin/bash
# -*-coding=utf-8-*-
import jieba
import re
from gensim.models import word2vec
import multiprocessing
import gensim


def segment_text(source_corpus, train_corpus, coding, punctuation):
    '''
    切词,去除标点符号
    :param source_corpus: 原始语料
    :param train_corpus: 切词语料
    :param coding: 文件编码
    :param punctuation: 去除的标点符号
    :return:
    '''
    with open(source_corpus, 'r', encoding=coding) as f, open(train_corpus, 'w', encoding=coding) as w:
        for line in f:
            # 去除标点符号
            line = re.sub('[{0}]+'.format(punctuation), '', line.strip())
            # 切词
            words = jieba.cut(line)
            w.write(' '.join(words))


if __name__ == '__main__':
    # 严格限制标点符号
    strict_punctuation = '。，、＇：∶；?‘’“”〝〞ˆˇ﹕︰﹔﹖﹑·¨….¸;！´？！～—ˉ｜‖＂〃｀@﹫¡¿﹏﹋﹌︴々﹟#﹩$﹠&﹪%*﹡﹢﹦﹤‐￣¯―﹨ˆ˜﹍﹎+=<­­＿_-\ˇ~﹉﹊（）〈〉‹›﹛﹜『』〖〗［］《》〔〕{}「」【】︵︷︿︹︽_﹁﹃︻︶︸﹀︺︾ˉ﹂﹄︼'
    # 简单限制标点符号
    simple_punctuation = '’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    # 去除标点符号
    punctuation = simple_punctuation + strict_punctuation

    # 文件编码
    coding = 'utf-8'
    # 原始语料
    source_corpus_text = 'source.txt'

    # 是每个词的向量维度
    size = 400
    # 是词向量训练时的上下文扫描窗口大小，窗口为5就是考虑前5个词和后5个词
    window = 5
    # 设置最低频率，默认是5，如果一个词语在文档中出现的次数小于5，那么就会丢弃
    min_count = 1
    # 是训练的进程数，默认是当前运行机器的处理器核数。
    workers = multiprocessing.cpu_count()
    # 切词语料
    train_corpus_text = 'words.txt'
    # w2v模型文件
    model_text = 'w2v_size_{0}.model'.format(size)

    # 切词 @TODO 切词后注释
    segment_text(source_corpus_text, train_corpus_text, coding, punctuation)

    # w2v训练模型 @TODO 训练后注释
    sentences = word2vec.Text8Corpus(train_corpus_text)
    model = word2vec.Word2Vec(sentences=sentences, size=size, window=window, min_count=min_count, workers=workers)
    model.save(model_text)

    # 加载模型
    model = gensim.models.Word2Vec.load(model_text)
    # print(model['运动会'])

    # 计算一个词的最近似的词，倒序
    # similar_words = model.most_similar('球队')
    # for word in similar_words:
    #     print(word[0], word[1])

    # 计算两词之间的余弦相似度
    # sim1 = model.similarity('运动会', '总成绩')
    # sim2 = model.similarity('排名', '运动会')
    # sim3 = model.similarity('展示', '学院')
    # sim4 = model.similarity('学院', '体育')
    # print(sim1)
    # print(sim2)
    # print(sim3)
    # print(sim4)

    # 计算两个集合之间的余弦似度
    list1 = ['运动会', '总成绩']
    list2 = ['排名', '运动会']
    list3 = ['学院', '体育']
    list_sim1 = model.n_similarity(list1, list2)
    print(list_sim1)
    list_sim2 = model.n_similarity(list1, list3)
    print(list_sim2)

    # 选出集合中不同类的词语
    list = ['队员', '足球比赛', '小组', '代表队']
    print(model.doesnt_match(list))
    list = ['队员', '足球比赛', '小组', '西瓜']
    print(model.doesnt_match(list))


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\SYX\AppData\Local\Temp\jieba.cache
Loading model cost 1.242 seconds.
Prefix dict has been built succesfully.


0.5334339
-0.020843875
小组
小组


C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:85: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:87: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:92: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
C:\Users\SYX\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\gensim\models\keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=

In [ ]:
import jieba

novel = open('E:/NLP/Task6/神雕侠侣.txt','r')
content=novel.read()
novel_segmented = open('E:/NLP/Task6/神雕侠侣_segmented.txt','w')

cutword = jieba.cut(content,cut_all=False)
seg = ' '.join(cutword).replace(',','').replace('。','').replace('“','').replace('”','').replace('：','').replace('…','')\
            .replace('！','').replace('？','').replace('~','').replace('（','').replace('）','').replace('、','').replace('；','')
print(seg,file=novel_segmented)

novel.close()
novel_segmented.close()

In [ ]:
from gensim.models import word2vec

# 训练word2vec模型，生成词向量
from gensim.models import word2vec

# 训练word2vec模型，生成词向量
s = word2vec.LineSentence('E:/NLP/Task6/神雕侠侣_segmented.txt')
# 这里如果报编码错误，只需将‘神雕侠侣_segmented.txt’用、notepad++打开，然后转码为对应编码即可。
model = word2vec.Word2Vec(s,size=20,window=5,min_count=5,workers=4)
model.save('E:/NLP/Task6/result.txt')

In [1]:
import gensim
from gensim.models import word2vec
model = gensim.models.Word2Vec.load('E:/NLP/Task6/result.txt')
print(model['杨过'])
print('杨过 和 小龙女 ：',model.similarity('杨过','小龙女'))
print('李莫愁 和 小龙女 ：',model.similarity('李莫愁','小龙女'))

[ 3.5629341e-01  1.1276431e+00 -3.4880743e+00  1.2330546e+00
  1.3405569e-01  1.0074255e+00  9.8335452e-04  1.3692881e+00
  5.1486641e-01 -1.9461825e+00  6.2180161e-01 -4.4074956e-01
  1.6870692e+00  1.5112236e+00  1.2047711e-02 -5.5651855e-01
 -3.3170119e-01 -1.3757578e-01 -3.0506107e-01  1.1388290e-01]
杨过 和 小龙女 ： 0.96552145
李莫愁 和 小龙女 ： 0.922824


C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
C:\Users\SYX\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
